In [1]:
import time
import pandas as pd
from algosdk import account, mnemonic, logic
from algosdk.future import transaction
from algosdk.v2client import algod, indexer
from pyteal import compileTeal, Mode, Approve

ALGOD_TOKEN = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"
TESTNET_ALGOD_RPC = "https://testnet-api.algonode.network"
TESTNET_INDEXER_RPC = "https://testnet-idx.algonode.network"

algod_client = algod.AlgodClient(ALGOD_TOKEN, TESTNET_ALGOD_RPC)
indexer_client = indexer.IndexerClient(ALGOD_TOKEN, TESTNET_INDEXER_RPC)
sp_func = algod_client.suggested_params
on_complete_param = transaction.OnComplete.NoOpOC
    
mnemonic_1 = open("../wallet_1").read().replace(',', ' ')
mnemonic_2 = open("../wallet_2").read().replace(',', ' ')
mnemonic_3 = open("../wallet_3").read().replace(',', ' ')

alice_private_key = mnemonic.to_private_key(mnemonic_1)
alice_address = account.address_from_private_key(alice_private_key)
bob_private_key = mnemonic.to_private_key(mnemonic_2)
bob_address = account.address_from_private_key(bob_private_key)
carol_private_key = mnemonic.to_private_key(mnemonic_3)
carol_address = account.address_from_private_key(carol_private_key)

print("Alice {} Balance: {}".format(alice_address, 
                algod_client.account_info(alice_address).get('amount') / 1e6))
print("Bob   {} Balance: {}".format(bob_address, 
                algod_client.account_info(bob_address).get('amount') / 1e6))
print("Carol {} Balance: {}".format(carol_address, 
                algod_client.account_info(carol_address).get('amount') / 1e6))

def submit_transaction(private_key: str, unsigned_txn: transaction.Transaction):
    signed_txn = unsigned_txn.sign(private_key)
    txid = algod_client.send_transaction(signed_txn)
    print("Signed transaction with txID: {}".format(txid))
    confirmed_txn = transaction.wait_for_confirmation(algod_client, txid, 3)
    print("Confirmed on round {}!".format(confirmed_txn['confirmed-round']))
    transaction_response = algod_client.pending_transaction_info(txid)
    return transaction_response

Alice GZ4IJXXNRFT23E6SLUOSSUWN2LUDFQTX4F6SXF5EP27LFWTOWHPFANLYIQ Balance: 24.915993
Bob   CIK3P7U4PZBJESWQ3XFDCZDXEZ3JZLWY6XZHY4A4KEJCA5SYYIPBT6W7Y4 Balance: 15.347008
Carol K7ZJP3J7SYYNB42DPZMRY56X32HVTYOOZRA2ASE2IND4LSQKCL3CE2Z2YU Balance: 8.989


## 1. Asset Creation and OptIn

In [2]:
# Creation
create_asset_tx = submit_transaction(alice_private_key, transaction.AssetCreateTxn(
    alice_address, sp_func(), int(1e18), 6, False, unit_name="mUSDC", asset_name="Meson Test Circle USD"
))
usdc_index = create_asset_tx['asset-index']
print("Asset [Meson Test Circle USD] created success! Asset id: %s\n" % usdc_index)

create_asset_tx = submit_transaction(alice_private_key, transaction.AssetCreateTxn(
    alice_address, sp_func(), int(1e18), 6, False, unit_name="mUSDT", asset_name="Meson Test Tether"
))
usdt_index = create_asset_tx['asset-index']
print("Asset [Meson Test Tether] created success! Asset id: %s\n" % usdt_index)

Signed transaction with txID: RXOM4DVKIEBUYX7FSPOHD2SQLPX5B6TCPHSJKS5L76XEK73UIZRA
Confirmed on round 27892638!
Asset [Meson Test Circle USD] created success! Asset id: 160363393

Signed transaction with txID: VQCAAMUL7KW5RTMRDCM6G5VRUVG4O2Q6B6KNXKB3VHNWIGODJPMQ
Confirmed on round 27892640!
Asset [Meson Test Tether] created success! Asset id: 160363405



In [3]:
# Optin
submit_transaction(bob_private_key, transaction.AssetOptInTxn(
    bob_address, sp_func(), usdc_index
)); print("Bob opt in mUSDC asset success!\n")

submit_transaction(carol_private_key, transaction.AssetOptInTxn(
    carol_address, sp_func(), usdc_index
)); print("Carol opt in mUSDC asset success!\n")

submit_transaction(bob_private_key, transaction.AssetOptInTxn(
    bob_address, sp_func(), usdt_index
)); print("Bob opt in mUSDT asset success!\n")

submit_transaction(carol_private_key, transaction.AssetOptInTxn(
    carol_address, sp_func(), usdt_index
)); print("Carol opt in mUSDT asset success!\n")

Signed transaction with txID: EWW3B5FIRFDM6WKADPHUSMM3OKCIQ5UCDKDROSAQPZJOHNANBKJA
Confirmed on round 27892658!
Bob opt in mUSDC asset success!

Signed transaction with txID: HEUXQVTMRGLTDS2ONSBRXPPQFVGFGU6FVKPQTPENV2MNNE4SLI7A
Confirmed on round 27892660!
Carol opt in mUSDC asset success!

Signed transaction with txID: O6CU4TVXM7UI5OZTCQLLY4XWNUQMXQ22RO3AWQE6OG4L7YA57GTQ
Confirmed on round 27892662!
Bob opt in mUSDT asset success!

Signed transaction with txID: U3ERKP5AH36ZZWJ7GKQCAUV6HPXAKOUUEPPCH2B6BWM45P4SRMGQ
Confirmed on round 27892664!
Carol opt in mUSDT asset success!



In [4]:
# Transfer
asset_amount = 2e12

submit_transaction(alice_private_key, transaction.AssetTransferTxn(
    alice_address, sp_func(), bob_address, int(asset_amount), usdc_index
)); print("Bob received %d mUSDC!" % (asset_amount / 1e6))

submit_transaction(alice_private_key, transaction.AssetTransferTxn(
    alice_address, sp_func(), carol_address, int(asset_amount), usdc_index
)); print("Carol received %d mUSDC!" % (asset_amount / 1e6))

submit_transaction(alice_private_key, transaction.AssetTransferTxn(
    alice_address, sp_func(), bob_address, int(asset_amount), usdt_index
)); print("Bob received %d mUSDT!" % (asset_amount / 1e6))

submit_transaction(alice_private_key, transaction.AssetTransferTxn(
    alice_address, sp_func(), carol_address, int(asset_amount), usdt_index
)); print("Carol received %d mUSDT!" % (asset_amount / 1e6))

Signed transaction with txID: LMB4X5LQBT4WFRQ7WJPN5MCGP5LUM37LTRDMKUV2FJCRJAQOWT3Q
Confirmed on round 27892666!
Bob received 2000000 mUSDC!
Signed transaction with txID: IIYMZA2RMIIG277HYAAZCZ6DGLMFYGDPJ3HXMKBYX2JGDK4ZH3HA
Confirmed on round 27892668!
Carol received 2000000 mUSDC!
Signed transaction with txID: UAY5MIOXBW5EFEFCP2TPEYSS4KAN7TRHJ6XCMVDUUSJ4EB3XBSPQ
Confirmed on round 27892670!
Bob received 2000000 mUSDT!
Signed transaction with txID: 6QZWBHX4TB22UQ4NL2KKZMBAIIBJYELKAVC7VNHK35AW7CCR5VAA
Confirmed on round 27892672!
Carol received 2000000 mUSDT!


## 2. Create Contract

In [6]:
usdc_index = 160363393
usdt_index = 160363405

import base64
from base64 import b64encode as en64
from base64 import b64decode as de64
import sys
sys.path.append('../contract')
from typing import List

def submit_transaction(private_key: str, unsigned_txn: transaction.Transaction):
    signed_txn = unsigned_txn.sign(private_key)
    txid = algod_client.send_transaction(signed_txn)
    print("Signed transaction with txID: {}".format(txid))
    confirmed_txn = transaction.wait_for_confirmation(algod_client, txid, 3)
    print("Confirmed on round {}!".format(confirmed_txn['confirmed-round']))
    transaction_response = algod_client.pending_transaction_info(txid)
    return transaction_response
    
def submit_transaction_group(private_key: str, unsigned_txns: List[transaction.Transaction]):
    gid = transaction.calculate_group_id(unsigned_txns)
    signed_txns = []
    for unsigned in unsigned_txns:
        unsigned.group = gid
        signed = unsigned.sign(private_key)
        signed_txns.append(signed)
    gtxid = algod_client.send_transactions(signed_txns)
    print("Signed transaction group with gtxID: {}".format(gtxid))
    confirmed_txn = transaction.wait_for_confirmation(algod_client, gtxid, 3)
    print("Confirmed on round {}!".format(confirmed_txn['confirmed-round']))
    transaction_response = algod_client.pending_transaction_info(gtxid)
    return transaction_response

def compile_program(client, source_code):
    compile_response = client.compile(source_code)
    return base64.b64decode(compile_response['result'])

blank_program = compile_program(algod_client, compileTeal(
    Approve(), Mode.Application, version=8
))

### 2.1 MesonSwap

In [7]:
# Create app and save some $ALGO
from MesonSwap import mesonSwapMainFunc

meson_swap = compile_program(algod_client, teal_sentences := compileTeal(
    mesonSwapMainFunc(), Mode.Application, version=8
))
create_app_tx = submit_transaction(alice_private_key, transaction.ApplicationCreateTxn(
    alice_address, sp_func(), on_complete_param, meson_swap, blank_program,
    transaction.StateSchema(12, 0), transaction.StateSchema(4, 0)       # todo: add variable nums
))
print("Create MesonSwap success! App id: %s, App Address: %s\n" % (
    mesonswap_index := create_app_tx['application-index'],
    mesonswap_address := logic.get_application_address(mesonswap_index)
))

send_amount = 400_000
transfer_algo_to_app_tx = submit_transaction(alice_private_key, transaction.PaymentTxn(
    alice_address, sp_func(), mesonswap_address, send_amount,
))
print("Transfer $ALGO to application address success!")
print("App   {} Balance: {} $ALGO\n".format(mesonswap_address, 
                algod_client.account_info(mesonswap_address).get('amount') / 1e6))

# Add USDC and USDT
submit_transaction(alice_private_key, transaction.ApplicationCallTxn(
    alice_address, sp_func(), mesonswap_index, on_complete_param,
    app_args=['addSupportToken', 1], foreign_assets=[usdc_index]
))
print("Mesonswap Optin USDC success!\n")
submit_transaction(alice_private_key, transaction.ApplicationCallTxn(
    alice_address, sp_func(), mesonswap_index, on_complete_param,
    app_args=['addSupportToken', 2], foreign_assets=[usdt_index]
))
print("Mesonswap Optin USDT success!\n")

Signed transaction with txID: V3YGCCIDYVEEUR6QFY5R4AW5EA6JUPX4MMVE6V7V7EZIZQNOMCLQ
Confirmed on round 27892691!
Create MesonSwap success! App id: 160363717, App Address: Y3FWJDCSYNKQ7RHDITJ27GTGZOLU2T6YPIH56X7V5DVQWETA2RZE5BE4CY

Signed transaction with txID: WAFE22BQO662EBMSRCRFJQ3GB462GIQTNFEF42CYQH7ORDTBN63A
Confirmed on round 27892693!
Transfer $ALGO to application address success!
App   Y3FWJDCSYNKQ7RHDITJ27GTGZOLU2T6YPIH56X7V5DVQWETA2RZE5BE4CY Balance: 0.4 $ALGO

Signed transaction with txID: UIOQ66KRLPVUZWQT3QTSPF5SA23GWRH4BODTFAQ5Q74EWYSVTIQQ
Confirmed on round 27892695!
Mesonswap Optin USDC success!

Signed transaction with txID: 4GXTZAB44ELVSE7MU72OI3QDXDZFZSFIG4GCJYUGGT5AHBBJBFCA
Confirmed on round 27892697!
Mesonswap Optin USDT success!



### 2.2 MesonPools

In [8]:
# Create app and save some $ALGO
from MesonPools import mesonPoolsMainFunc

meson_pools = compile_program(algod_client, teal_sentences := compileTeal(
    mesonPoolsMainFunc(), Mode.Application, version=8
))
create_app_tx = submit_transaction(alice_private_key, transaction.ApplicationCreateTxn(
    alice_address, sp_func(), on_complete_param, meson_pools, blank_program,
    transaction.StateSchema(12, 0), transaction.StateSchema(4, 0)       # todo: add variable nums
))
print("Create MesonPools success! App id: %s, App Address: %s\n" % (
    mesonpools_index := create_app_tx['application-index'],
    mesonpools_address := logic.get_application_address(mesonpools_index)
))

send_amount = 400_000
transfer_algo_to_app_tx = submit_transaction(alice_private_key, transaction.PaymentTxn(
    alice_address, sp_func(), mesonpools_address, send_amount,
))
print("Transfer $ALGO to application address success!")
print("App   {} Balance: {} $ALGO\n".format(mesonpools_address, 
                algod_client.account_info(mesonpools_address).get('amount') / 1e6))

# Add USDC and USDT
submit_transaction(alice_private_key, transaction.ApplicationCallTxn(
    alice_address, sp_func(), mesonpools_index, on_complete_param,
    app_args=['addSupportToken', 1], foreign_assets=[usdc_index]
))
print("Mesonpools Optin USDC success!\n")
submit_transaction(alice_private_key, transaction.ApplicationCallTxn(
    alice_address, sp_func(), mesonpools_index, on_complete_param,
    app_args=['addSupportToken', 2], foreign_assets=[usdt_index]
))
print("Mesonpools Optin USDT success!\n")

Signed transaction with txID: OSKHQPRRDIDOHUA4OWYVCWKA3FAYTXW2XNRFAEK24JECVZMXGBBQ
Confirmed on round 27892707!
Create MesonPools success! App id: 160363828, App Address: 7EPQEQRPU7KNKO4KQOE37TYRU3DIVVH6JZWNSQT6ZJOZFWT3EE3NXEVPD4

Signed transaction with txID: FPIWB3C5OLODIG2IW4TAIWE67XQEAUYZF3C6QY3UFT6NSVVHK55A
Confirmed on round 27892709!
Transfer $ALGO to application address success!
App   7EPQEQRPU7KNKO4KQOE37TYRU3DIVVH6JZWNSQT6ZJOZFWT3EE3NXEVPD4 Balance: 0.4 $ALGO

Signed transaction with txID: H4D6UAM25Q3OSSPTITYJLHB77UTFKZ4VTFPMKSDNAZNHSCA5SFWQ
Confirmed on round 27892711!
Mesonpools Optin USDC success!

Signed transaction with txID: AIUDYBU7RK26ZXH6QOYQS7D2OJ2M4COQXXSBTCLMCQYWMSERMJWQ
Confirmed on round 27892713!
Mesonpools Optin USDT success!



## 3. LP deposit some assets

### 3.1 LP Opt in Meson Pools

In [9]:
submit_transaction(bob_private_key, transaction.ApplicationOptInTxn(
    bob_address, sp_func(), mesonpools_index
))
print("LP(Bob) opt in MesonPools app success!")

Signed transaction with txID: EXBHDKQ5XHI7X4CJQ775GF2JM4CCILASFATKKAYAUQQASZNNYNCA
Confirmed on round 27892718!
LP(Bob) opt in MesonPools app success!


### 3.2 LP Deposit

In [10]:
amount_deposit = 120 * 1_000_000

submit_transaction_group(bob_private_key, [
    transaction.ApplicationCallTxn(
        bob_address, sp_func(), mesonpools_index, on_complete_param,
        app_args=['deposit', amount_deposit],
        foreign_assets=[usdc_index],
    ), 
    transaction.AssetTransferTxn(
        bob_address, sp_func(), mesonpools_address, amount_deposit, usdc_index,
    )
])
print("LP(Bob) deposit %d mUSDC into MesonPools!\n" % (amount_deposit // 1e6))

submit_transaction_group(bob_private_key, [
    transaction.ApplicationCallTxn(
        bob_address, sp_func(), mesonpools_index, on_complete_param,
        app_args=['deposit', amount_deposit],
        foreign_assets=[usdt_index],
    ), 
    transaction.AssetTransferTxn(
        bob_address, sp_func(), mesonpools_address, amount_deposit, usdt_index,
    )
])
print("LP(Bob) deposit %d mUSDT into MesonPools!\n" % (amount_deposit // 1e6))

returned = indexer_client.lookup_account_application_local_state(bob_address, application_id=mesonpools_index)
balances_saved = returned['apps-local-states'][0]['key-value']
print("LP(Bob) balance:")
for balance in balances_saved:
    print("Asset %d: %d" % (int.from_bytes(de64(balance['key'])[8:], 'big'), balance['value']['uint'] / 1e6))

Signed transaction group with gtxID: TSD3QYYLJ7TGZOEVJBPCC5IUCNCCJTTSIZVTD3YBXIXVA5XQMM3Q
Confirmed on round 27892720!
LP(Bob) deposit 120 mUSDC into MesonPools!

Signed transaction group with gtxID: AKLMC2JOUKZCFCJ4DBJ5GEA7DCY2JYFZURHEBMLABEEYIUDQXUPA
Confirmed on round 27892722!
LP(Bob) deposit 120 mUSDT into MesonPools!

LP(Bob) balance:
Asset 160363393: 120
Asset 160363405: 120


### 3.3 LP Withdraw

In [53]:
amount_withdraw = 30 * 1_000_000

submit_transaction(bob_private_key, transaction.ApplicationCallTxn(
    bob_address, sp_func(), mesonpools_index, on_complete_param,
    app_args=['withdraw', amount_withdraw, usdt_index],
    foreign_assets=[usdt_index],
))
print("LP(Bob) withdraw %d mUSDT from MesonPools!\n" % (amount_withdraw // 1e6))

returned = indexer_client.lookup_account_application_local_state(bob_address, application_id=mesonpools_index)
balances_saved = returned['apps-local-states'][0]['key-value']
print("LP(Bob) balance:")
for balance in balances_saved:
    print("Asset %d: %d" % (int.from_bytes(de64(balance['key'])[8:], 'big'), balance['value']['uint'] / 1e6))

Signed transaction with txID: LCCR3UPSGOQTEJPX5OJ3244HF5XNEVY7YZONSYJFHXBEHAIPIPFQ
Confirmed on round 27842562!
LP(Bob) withdraw 30 mUSDT from MesonPools!

LP(Bob) balance:
Asset 159625952: 120
Asset 159625960: 330


## 4. Start cross-chain swap

### 4.0 Build EncodedSwap

In [12]:
# origin format: [bytes32]
# \x01\x00\x01\xe8H\x00\xc0\x00\x00\x00\x00\x00\xe7U& \x00\x00\x00\x00\x00\x00c\xd5\x00B#)\x02\x02\xca
# hex format [uint256, hex64]:
# (0x)010001e84800c00000000000e755262000000000000063d5004223290202ca22
# split values: 0x|01|0001e84800|c00000000000e7552620|0000000000|0063d50042|2329|02|02ca|22
# split variables: 0x|version|amount|salt|fee|expireTs|outChain|outToken|inChain|inToken
# salt[0xc0, 0b11000000] -> [(0)willTransferToContract, (1)feeWaived, -, -, (1)signNonTyped, -, -, -]

def get_expire_ts(delay=90):   # default to 90 minutes
    return int(time.time()) + 60*delay

def build_encoded(amount: int, expireTs: int, outToken, inToken, 
                  salt='c00000000000e7552620', fee='0000000000', return_bytes=True):
    assert amount < 0x1111111111
    version = '01'
    amount_string = hex(amount)[2:].rjust(10, '0')
    expireTs_string = hex(expireTs)[2:].rjust(10, '0')
    outChain = '011b'
    inChain = '011b'
    encoded_string = ''.join([
        '0x', version, amount_string, salt, fee, expireTs_string, outChain, outToken, inChain, inToken
    ])
    return bytes.fromhex(encoded_string[2:]) if return_bytes else encoded_string

'''
ZERO_ADDRESS_STR = '00' * 32
def build_posted(initiator, tokenIndex, lp=ZERO_ADDRESS_STR, return_bytes=True):
    assert len(initiator) == len(lp) == 64
    posted_string = initiator + lp + tokenIndex
    return bytes.fromhex(posted_string[2:]) if return_bytes else posted_string
'''

mesonswap_index = 160363717
mesonswap_address = logic.get_application_address(mesonswap_index)
mesonpools_index = 160363828
mesonpools_address = logic.get_application_address(mesonpools_index)

In [6]:
# Transfer to contract if needed
send_amount = 400_000
transfer_algo_to_app_tx = submit_transaction(alice_private_key, transaction.PaymentTxn(
    alice_address, sp_func(), mesonswap_address, send_amount,
))
print("Transfer $ALGO to application address success!")
print("App   {} Balance: {} $ALGO\n".format(mesonswap_address, 
                algod_client.account_info(mesonswap_address).get('amount') / 1e6))

Signed transaction with txID: 6FLJ2NKJU3ZSS4OP35NNLMLVOIIQGX5AG5KOQABS7O6YNBP23W7Q
Confirmed on round 27853155!
Transfer $ALGO to application address success!
App   A4VCUJKIXTWUNY2DPTLBSVGXL7OQK4QQVVGLZMKXR3OXOZPHNQW5AJ5JNQ Balance: 0.798 $ALGO



In [18]:
amount_transfer = 50 * 1_000_000
encodedSwap = build_encoded(amount_transfer, get_expire_ts(), '02', '01')

### 4.1 postSwap & bondSwap

In [19]:
submit_transaction_group(carol_private_key, [
    transaction.ApplicationCallTxn(
        carol_address, sp_func(), mesonswap_index, on_complete_param,
        app_args=['postSwap', encodedSwap, 1111, 1111],
        boxes=[(mesonswap_index, encodedSwap)]
    ),
    transaction.AssetTransferTxn(
        carol_address, sp_func(), mesonswap_address, amount_transfer, usdc_index
    )
])
print("Step 1.1. User(Carol) posted swap success!\n")

Signed transaction group with gtxID: W6LIJCIUS7AA3LXFM3BUNMH3QJM2ZGXTD7J4IWCFLG5TDDPFAKGQ
Confirmed on round 27892759!
Step 1.1. User(Carol) posted swap success!



In [20]:
submit_transaction(bob_private_key, transaction.ApplicationCallTxn(
    bob_address, sp_func(), mesonswap_index, on_complete_param,
    app_args=['bondSwap', encodedSwap],
    boxes=[(mesonswap_index, encodedSwap)]
))
print("Step 1.2. LP(Bob) Bonded swap success!")

Signed transaction with txID: IYIHS4VVNFLI3REL5RJUGUQKWJAGRIUHLYYGJ6NTVD7NNK243AFQ
Confirmed on round 27892762!
Step 1.2. LP(Bob) Bonded swap success!


In [21]:
print("Mesonswap Boxes: ")
for name_dict in algod_client.application_boxes(mesonswap_index)['boxes']:
    encoded_key = base64.b64decode(name_dict['name'].encode())
    value_return = algod_client.application_box_by_name(mesonswap_index, encoded_key)
    posted_value = base64.b64decode(value_return['value'])
    print('[EncodedSwap] %s, \n\t[PostedValue] (%s, \n\t\t\t%s, %s)' % (encoded_key.hex(), 
          (p := posted_value.hex())[:64], p[64:128], p[128:]))

Mesonswap Boxes: 
[EncodedSwap] 010002faf080c00000000000e755262000000000000063f477ef011b02011b01, 
	[PostedValue] (57f297ed3f9630d0f3437e591c77d7de8f59e1cecc41a0489a4347c5ca0a12f6, 
			1215b7fe9c7e42924ad0ddca31647726769caed8f5f27c701c5112207658c21e, 01)
[EncodedSwap] 010002faf080c00000000000e755262000000000000063f477b8011b02011b01, 
	[PostedValue] (57f297ed3f9630d0f3437e591c77d7de8f59e1cecc41a0489a4347c5ca0a12f6, 
			0000000000000000000000000000000000000000000000000000000000000000, 01)


### 4.2 lock

In [22]:
from Crypto.Hash import keccak

def get_swapID(encodedSwap, initiator_base32_address, return_bytes=True):
    concat = encodedSwap + base64.b32decode(initiator_base32_address + '======')[:-4]
    assert len(concat) == 64
    keccak_func = keccak.new(digest_bits=256)
    hash_value = keccak_func.update(concat).hexdigest()
    return bytes.fromhex(hash_value) if return_bytes else hash_value
    
get_swapID(encodedSwap, carol_address)

b'$\x93\xf4\x16\xbcU\x0c\xd4\x96\xa7\x1bC\xee\xb6\xe4\xd4\xb3zJ\x9c\xa5S\x149~\x85R\xf4"7\xcb]'

In [23]:
submit_transaction(bob_private_key, transaction.ApplicationCallTxn(
    bob_address, sp_func(), mesonpools_index, on_complete_param,
    app_args=['lock', encodedSwap, 1111, 1111],
    accounts=[carol_address],
    boxes=[(mesonpools_index, get_swapID(encodedSwap, carol_address))]
))
print("Step 2. LP(Bob) lock assets success!\n")

print("Mesonpools Boxes: ")
for name_dict in algod_client.application_boxes(mesonpools_index)['boxes']:
    swapid_key = base64.b64decode(name_dict['name'].encode())
    value_return = algod_client.application_box_by_name(mesonpools_index, swapid_key)
    locked_value = base64.b64decode(value_return['value'])
    print('[SwapID] %s, \n\t[LockedValue] (%s, \n\t\t\t%s, %s)' % (swapid_key.hex(), 
          pd.to_datetime(int((p := locked_value.hex())[:10], 16) * 1e9), p[10:74], p[74:]))

Signed transaction with txID: ZCSSDE4UO5R6E54TSRYZPNO3I4P3UCKLDYW7YT5ISXWVNU3GC67Q
Confirmed on round 27892782!
Step 2. LP(Bob) lock assets success!

Mesonpools Boxes: 
[EncodedSwap] 2493f416bc550cd496a71b43eeb6e4d4b37a4a9ca55314397e8552f42237cb5d, 
	[LockedValue] (2023-02-21 06:42:27, 
			1215b7fe9c7e42924ad0ddca31647726769caed8f5f27c701c5112207658c21e, 02)


### 4.3 release

In [24]:
submit_transaction(carol_private_key, transaction.ApplicationCallTxn(
    carol_address, sp_func(), mesonpools_index, on_complete_param,
    app_args=['release', encodedSwap, 1111, 1111],
    accounts=[carol_address, carol_address],
    foreign_assets=[usdt_index],
    boxes=[(mesonpools_index, get_swapID(encodedSwap, carol_address))]
))
print("Step 3. User(Carol) release assets success!")

Signed transaction with txID: PZJXVC57Y3AXORLBQ4QKSFOUPXCCZ6WQZ65QVJJKB67XHGSXZEOA
Confirmed on round 27892791!
Step 3. User(Carol) release assets success!


In [25]:
print("Mesonpools Boxes: ")
for name_dict in algod_client.application_boxes(mesonpools_index)['boxes']:
    swapid_key = base64.b64decode(name_dict['name'].encode())
    value_return = algod_client.application_box_by_name(mesonpools_index, swapid_key)
    locked_value = base64.b64decode(value_return['value'])
    print('[EncodedSwap] %s, \n\t[LockedValue] (%s, \n\t\t\t%s, %s)' % (swapid_key.hex(), 
          pd.to_datetime(int((p := locked_value.hex())[:10], 16) * 1e9), p[10:74], p[74:]))

Mesonpools Boxes: 
[EncodedSwap] 2493f416bc550cd496a71b43eeb6e4d4b37a4a9ca55314397e8552f42237cb5d, 
	[LockedValue] (NaT, 
			1111111111111111111111111111111111111111111111111111111111111111, 11)


### 4.4 executeSwap

In [26]:
submit_transaction(bob_private_key, transaction.ApplicationCallTxn(
    bob_address, sp_func(), mesonswap_index, on_complete_param,
    app_args=['executeSwap', encodedSwap, 1111, 1111, 0],
    accounts=[carol_address],
    foreign_assets=[usdc_index],
    boxes=[(mesonswap_index, encodedSwap)]
))
print("Step 4. LP(Bob) executeSwap success!")

Signed transaction with txID: GGO2MGFJY637723GBSNUGSM6AGBJQJX3S5AZ3K4PYKSRB7HQ5J6A
Confirmed on round 27892797!
Step 4. LP(Bob) executeSwap success!


In [27]:
print("Mesonswap Boxes: ")
for name_dict in algod_client.application_boxes(mesonswap_index)['boxes']:
    encoded_key = base64.b64decode(name_dict['name'].encode())
    value_return = algod_client.application_box_by_name(mesonswap_index, encoded_key)
    posted_value = base64.b64decode(value_return['value'])
    print('[EncodedSwap] %s, \n\t[PostedValue] (%s, \n\t\t\t%s, %s)' % (encoded_key.hex(), 
          (p := posted_value.hex())[:64], p[64:128], p[128:]))

Mesonswap Boxes: 
[EncodedSwap] 010002faf080c00000000000e755262000000000000063f477ef011b02011b01, 
	[PostedValue] (1111111111111111111111111111111111111111111111111111111111111111, 
			1111111111111111111111111111111111111111111111111111111111111111, 11)
[EncodedSwap] 010002faf080c00000000000e755262000000000000063f477b8011b02011b01, 
	[PostedValue] (57f297ed3f9630d0f3437e591c77d7de8f59e1cecc41a0489a4347c5ca0a12f6, 
			0000000000000000000000000000000000000000000000000000000000000000, 01)


In [14]:
carol_address, bob_address

('K7ZJP3J7SYYNB42DPZMRY56X32HVTYOOZRA2ASE2IND4LSQKCL3CE2Z2YU',
 'CIK3P7U4PZBJESWQ3XFDCZDXEZ3JZLWY6XZHY4A4KEJCA5SYYIPBT6W7Y4')